# Problem 2 - Working with SparkSQL (5 points)

This is an interactive PySpark session. Remember that when you open this notebook the `SparkContext` and `SparkSession` are already created, and they are in the `sc` and `spark` variables, respectively. You can run the following two cells to make sure that the Kernel is active.

**Do not insert any additional cells than the ones that are provided.**

In [ ]:
sc

In [ ]:
spark

## Quazyilx again!

Yes, you remember it. As a reminder, here is the description of the files.

The quazyilx has been malfunctioning, and occasionally generates output with a `-1` for all four measurements, like this:

    2015-12-10T08:40:10Z fnard:-1 fnok:-1 cark:-1 gnuck:-1

There are four different versions of the _quazyilx_ file, each of a different size. As you can see in the output below the file sizes are 50MB (1,000,000 rows), 4.8GB (100,000,000 rows), 18GB (369,865,098 rows) and 36.7GB (752,981,134 rows). The only difference is the length of the number of records, the file structure is the same.

```
[hadoop@ip-172-31-1-240 ~]$ hadoop fs -ls s3://bigdatateaching/quazyilx/
Found 4 items
-rw-rw-rw-   1 hadoop hadoop    52443735 2018-01-25 15:37 s3://bigdatateaching/quazyilx/quazyilx0.txt
-rw-rw-rw-   1 hadoop hadoop  5244417004 2018-01-25 15:37 s3://bigdatateaching/quazyilx/quazyilx1.txt
-rw-rw-rw-   1 hadoop hadoop 19397230888 2018-01-25 15:38 s3://bigdatateaching/quazyilx/quazyilx2.txt
-rw-rw-rw-   1 hadoop hadoop 39489364082 2018-01-25 15:41 s3://bigdatateaching/quazyilx/quazyilx3.txt
```

You will use Spark and SparkSQL to create a Spark DataFrame and then run some analysis on the files using SparkSQL queries.

First, in the following cell, create an RDD called `quazyilx` that reads the `quazyilx1.txt` file from S3.

In [ ]:
quazyilx=sc.textFile("s3://bigdatateaching/quazyilx/quazyilx1.txt")

In the next cell, look at the first 50 elements of `quazyilx` to make sure everything is working corectly. This should take a few seconds.

In [ ]:
quazyilx.take(50)

We now need to work with the RDD to be able to make a DataFrame. In the following cell, modify the code to create a class called `quazyilx_class` that processes a line and returns it as a slotted structure, with attributes for the `.time`, `.fnard`, `.fnok` and `.cark`. 

You will need to define the Regular Expression and complete the class where it says `#Put your code here:`

In [60]:
import sys
import os,datetime,re

QUAZYILX_RE = "(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}) \w{5}:(-?\d+) \w{4}:(-?\d+) \w{4}:(-?\d+) \w{5}:(-?\d+)"
quazyilx_re = re.compile(QUAZYILX_RE)

class quazyilx_class():
    __slots__ = ['datetime','fnard','fnok','cark','gnuck']
    def __init__(self,line):
        # Parse line with the regular expression
        m = quazyilx_re.search(line)
        if not m:
            self.datetime = m.group(1)
            self.fnard    = m.group(2)
            self.fnok     = m.group(3)
            self.cark     = m.group(4)
            self.gnuck    = m.group(5)
            return 
        # Put your code here:
    def __str__(self):
        return "{} fnard:{} fnok:{} cark:{} gnuck:{}".format(self.datetime,self.fnard,self.fnok,self.cark,self.gnuck)
    def __repr__(self):
        return "{} fnard:{} fnok:{} cark:{} gnuck:{}".format(self.datetime,self.fnard,self.fnok,self.cark,self.gnuck)


You will then need to turn the quazyilx RDD into a `Row()` object. [Look at the notebook we used in Lab08 for reference](https://github.com/gu-anly502/lab08-spring-2018/blob/master/social-characteristics-of-the-marvel-universe.ipynb). You can do that with a lambda function, like this:

```(python)
lambda q:Row(datetime=q.datetime.isoformat(),fnard=q.fnard,fnok=q.fnok,cark=q.cark,gnuck=q.gnuck))
```

Alternatively, you can add a new method to the Quazyilx class called `.Row()` that returns a Row. All of these ways are more or less equivalent. You just need to pick one of them.  You may find it useful to look at [this documentation](http://spark.apache.org/docs/latest/sql-programming-guide.html#inferring-the-schema-using-reflection).

In the next cell, create an RDD called `line` that converts the `quazyilx` RDD into a `Row()` object using the `quazyilx_class`.

In [ ]:
line=quazyilx.map(lambda i:quazyilx_class(line=i))\
            .map(lambda q:Row(datetime=q.datetime.isoformat(),fnard=q.fnard,fnok=q.fnok,cark=q.cark,gnuck=q.gnuck))

Look at the first 10 rows to make sure everything is working.

In [ ]:
line.take(10)

In the following cell, convert the quazyilx RDD into a DataFrame `quazyilx_df` using the `spark.createDataFrame` method, register it as the SQL table `quazyilx_tbl` with the method `.createOrReplaceTempView`. You will want to cache the DataFrame so it doesn't get generated every time you run a query.

In [ ]:
quazyilx_df=spark.createDataFrame(quazyilx.map(lambda i:quazyilx_class(line=i))\
                                  .map(lambda q:Row(datetime=q.datetime.isoformat(),\
                                                    fnard=q.fnard,fnok=q.fnok,cark=q.cark,gnuck=q.gnuck)))

quazyilx_df.createOrReplaceTempView("quazyilx_tbl").cache()

Once you create and register the dataframe and table, you will run SQL queries using  `spark.sql()` to calculate the following:

1. The number of rows in the dataset
1. The number of lines that has -1 for `fnard`, `fnok`, `cark` and `gnuck`.
1. The number of lines that have -1 for `fnard` but have `fnok > 5` and `cark > 5`
1. The first datetime in the dataset
1. The first datetime that has -1 for all of the values
1. The last datetime in the dataset
1. The last datetime that has a -1 for all of the values

Place each query into each of the following  seven cells and run it to get the results. Remember, running the query statement itselft will not give you the results you want. You need to do something else to "get" the result.

**Note: in development testing, the first query may take approximately 10-15 minutes to run with the cluster configuration for this assignment (1 master, 4 task nodes of m4.xlarge). If you cache() correctly, all subsequent queries should take no more than 5 seconds.**


In [ ]:
df1=spark.sql("""SELECT COUNT(*)
FROM quazyilx_tbl""").cache()
df1.take()

In [ ]:
df2=spark.sql("""SELECT COUNT(*)
FROM(
SELECT q.fnard, q.fnok, q.cark, q.gnuck
FROM quazyilx_tbl AS q
WHERE q.fnard=-1 AND q.fnok=-1 AND q.cark=-1 AND q.gnuck=-1
) AS r1""").cache()
df2.take()

In [ ]:
df3=spark.sql("""SELECT COUNT(*)
FROM(
SELECT q.fnard, q.fnok, q.cark, q.gnuck
FROM quazyilx_tbl AS q
WHERE q.fnard=-1 AND q.fnok>5 AND q.cark>5
) AS r2""").cache()
df3.take()

In [ ]:
df4=spark.sql("""SELECT MIN(q.datetime)
FROM quazyilx_tbl AS q
""").cache()
df4.take()

In [ ]:
df5=spark.sql("""SELECT MIN(r3.datetime)
FROM(
SELECT q.datetime
FROM quazyilx_tbl AS q
WHERE q.fnard=-1 AND q.fnok=-1 AND q.cark=-1 AND q.gnuck=-1
) AS r3""").cache()
df5.take()

In [ ]:
df6=spark.sql("""SELECT MAX(q.datetime)
FROM quazyilx_tbl AS q
""").cache()
df6.take()

In [ ]:
df7=spark.sql("""SELECT MAX(r4.datetime)
FROM(
SELECT q.datetime
FROM quazyilx_tbl AS q
WHERE q.fnard=-1 AND q.fnok=-1 AND q.cark=-1 AND q.gnuck=-1
) AS r4""").cache()
df7.take()

When you finish this problem, click on the File -> 'Save and Checkpoint' in the menu bar to make sure that the latest version of the workbook file is saved. Also, before you close this notebook and move on, make sure you disconnect your SparkContext, otherwise you will not be able to re-allocate resources. Remember, you will commit the .ipynb file to the repository for submission (in the master node terminal.)

In [ ]:
sc.stop()